In [1]:
import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split

print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

import numpy
import numpy as np

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

import pandas
import pandas as pd

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
import os
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

__Python VERSION: 3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
__pyTorch VERSION: 1.2.0
__CUDA VERSION
/bin/sh: 1: nvcc: not found
__CUDNN VERSION: 7301
__Number CUDA Devices: 2
__Devices
Active CUDA Device: GPU 0
Available devices  2
Current cuda device  0
3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
9.9
svmem(total=67477729280, available=14291292160, percent=78.8, used=51882876928, free=6504308736, active=47740014592, inactive=6667427840, buffers=90071040, cached=9000472576, shared=535216128, slab=4716032000)
memory GB: 0.24835968017578125


In [2]:
use_cuda = torch.cuda.is_available()
# use_cuda = False

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

In [3]:
os.chdir('data')

In [4]:
!pwd

/projects/audio-pipelines/data


In [77]:
hyps = pd.read_csv("sets/full_set/hyps", names=['audio_id'])
hyps[['audio_id','hyps']] = hyps["audio_id"].str.split(" ", 1, expand=True)

targets = pd.read_csv("sets/full_set/targets", names=['audio_id'])
targets[['audio_id','target']] = targets["audio_id"].str.split(" ", 1, expand=True)

text = pd.read_csv("sets/full_set/text", names=['audio_id'])
text[['audio_id','text']] = text["audio_id"].str.split(" ", 1, expand=True)

speakers = pd.read_csv("sets/full_set/utt2spk", names=['audio_id'])
speakers[['audio_id','speaker']] = speakers["audio_id"].str.split(" ", 1, expand=True)

images = pd.read_csv("sets/full_set/wav.scp", names=['audio_id'])
images[['audio_id','audio_path']] = images["audio_id"].str.split(" ", 1, expand=True)

dataset = pd.merge(hyps, text, how="left")
dataset = pd.merge(dataset, speakers, how="left")
dataset = pd.merge(dataset, images, how="left")
dataset = pd.merge(dataset, targets, how="left")

In [133]:
dataset['image_path'] = dataset['audio_path'].apply(lambda x : x[:-4] + ".png")
def remove_absolute(string, prefix='/home/raznem/projects/audio-pipelines/data/'):
    if string.startswith(prefix):
        string = string[len(prefix):]
    return string
    
dataset['image_path'] = dataset['image_path'].apply(remove_absolute)

In [134]:
dataset.head()

,audio_id,hyps,text,speaker,audio_path,target,image_path
0,Adam_Abramowicz_11_45_8-6,prog prezydenta powinien być przez państwa tyl...,krok prezydenta powinien być przez państwa tyl...,Adam_Abramowicz,data/sejm_voicelab/train/Adam_Abramowicz_11_45...,1,data/sejm_voicelab/train/Adam_Abramowicz_11_45...
1,Adam_Abramowicz_1_39_2-0,panie marszałku wysoki sejmie panie ministrze ...,panie marszałku wysoki sejmie panie ministrze ...,Adam_Abramowicz,data/sejm_voicelab/train/Adam_Abramowicz_1_39_...,1,data/sejm_voicelab/train/Adam_Abramowicz_1_39_...
2,AdamAbramowicz-20130410-file000,panie marszałku wysoka izbo panie ministrze pr...,panie marszałku wysoka izbo panie ministrze pr...,AdamAbramowicz,/home/raznem/projects/audio-pipelines/data/dat...,1,data/parlament_audio/SejmSenat/audio/AdamAbram...
3,AdamAbramowicz-20130410-file001,w grudniu kanada wystąpiła z protokółów z kiot...,w grudniu kanada wystąpiła z protokołu z kioto...,AdamAbramowicz,/home/raznem/projects/audio-pipelines/data/dat...,0,data/parlament_audio/SejmSenat/audio/AdamAbram...
4,AdamAbramowicz-20130410-file002,panie marszałku wysoka izbo panie ministrze sw...,panie marszałku wysoka izbo panie ministrze w ...,AdamAbramowicz,/home/raznem/projects/audio-pipelines/data/dat...,0,data/parlament_audio/SejmSenat/audio/AdamAbram...


In [135]:
!pwd

/projects/audio-pipelines/data


In [136]:
dataset.to_csv('data/dataset.csv')

In [137]:
image_path = dataset['image_path'][0]
image = Image.open(image_path)
image = image.convert('RGB')

In [138]:
type(image)

PIL.Image.Image

In [101]:
!pwd

/projects/audio-pipelines/data


In [4]:
from allennlp.commands.elmo import ElmoEmbedder
from torchvision import transforms

try:
    from PIL import Image
except ImportError:
    import Image
    
toImg = transforms.ToPILImage()
toTensor = transforms.ToTensor()
    
class WavImagesLoader(Dataset):
    def __init__(self, csv_path, transform=None):
        self.dataset = pd.read_csv(csv_path)
        self.transform = transform
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        return image, target
    
    def __len__(self):
        return len(self.dataset)
    
    @staticmethod        
    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))    

#     @staticmethod    
#     def flaotTensorToImage(img, mean=0, std=1):
#         """convert a tensor to an image"""
#         img = np.transpose(img.numpy(), (1, 2, 0))
#         img = (img*std+ mean)*255
#         img = img.astype(np.uint8)    
#         return img    
    
    
class ElmoWavImagesLoader(WavImagesLoader):
    def __init__(self, csv_path, elmo_model, transform=None):
        super().__init__(csv_path, transform=None)
        self.elmo_model = elmo_model
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(path)
        image = img.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
            
        text = self.dataset['hyps'][key]
        text = text.split(' ')
        embedding = self.elmo_model(text)
        embedding = embedding.sum(axis=0)

        return image, embedding, target
    
    
class ElmoWavVecLoader(ElmoWavImagesLoader):
    def __init__(self, csv_path, elmo_model, transform=None):
        super().__init__(csv_path, elmo_model, transform=None)
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_vec = self.dataset['image_vec'][key]
        text = self.dataset['hyps'][key]
        text = text.split(' ')
        embedding = self.elmo_model(text)
        embedding = embedding.sum(axis=0)

        return image_vec, embedding, target

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.modeling_bert:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.modeling_xlnet:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>


## Conv net to get vector from Image

In [5]:
dropout = torch.nn.Dropout(p=0.30)
class ConvRes(nn.Module):
    def __init__(self, insize, outsize):
        super(ConvRes, self).__init__()
        drate = .3
        self.math = nn.Sequential(
            nn.BatchNorm2d(insize),
            # nn.Dropout(drate),
            torch.nn.Conv2d(insize, outsize, kernel_size=2, padding=2),
            nn.PReLU(),
        )

    def forward(self, x):
        return self.math(x)


class ConvCNN(nn.Module):
    def __init__(self, insize, outsize, kernel_size=7, padding=2, pool=2, avg=True):
        super(ConvCNN, self).__init__()
        self.avg = avg
        self.math = torch.nn.Sequential(
            torch.nn.Conv2d(insize, outsize, kernel_size=kernel_size, padding=padding),
            torch.nn.BatchNorm2d(outsize),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(pool, pool),
        )
        self.avgpool = torch.nn.AvgPool2d(pool, pool)

    def forward(self, x):
        x = self.math(x)
        if self.avg is True:
            x = self.avgpool(x)
        return x


class Net(nn.Module):
    def __init__(self, output_dim):
        super(Net, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d(1)

        self.cnn1 = ConvCNN(3, 32, kernel_size=7, pool=4, avg=False)
        self.cnn2 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)
        self.cnn3 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)

        self.res1 = ConvRes(32, 64)

        self.features = nn.Sequential(
            self.cnn1, dropout,
            self.cnn2,
            self.cnn3,
            self.res1,
        )

        self.classifier = torch.nn.Sequential(
            nn.Linear(5184, output_dim),
        )
        self.sig = nn.Sigmoid()
  
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = self.sig(x)
        return x

    
# if use_cuda:
#     lgr.info ("Using the GPU")
#     model = Net(output_dim).cuda() # On GPU
# else:
#     lgr.info ("Using the CPU")
#     model = Net(output_dim) # On CPU

# lgr.info('Model {}'.format(model))

In [6]:
import gc
def image_trainer(loader_train, model, optimizer, exp_name='', save_every=None, print_every=None, epochs=1, use_gpu=True, 
                 dtype=torch.float32):
    if use_gpu and torch.cuda.is_available():
        device = torch.device('cuda:1')
    else:
        device = torch.device('cpu')
        
    model.train()
    model.to(device=device)

    for e in range(epochs):
        print('Epoch %d' %e)
        for t, (x, y) in enumerate(loader_train):
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=dtype)

            y_pred = model(x)
            y_pred = torch.squeeze(y_pred)
            loss = loss_fn(y_pred, y)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            if print_every is not None and t % print_every == 0:
                print('Iteration %d, loss = %.4f; epoch %d' % (t, loss.item(), e))
                
        if save_every is not None and e % save_every == 0:
            torch.save(model.state_dict(), f'models/{exp_name}_e%d_image_cnn.pt' % e)
            gc.collect()
    torch.save(model.state_dict(), f'models/{exp_name}_image_cnn.pt')

In [7]:
loss_fn = torch.nn.BCELoss()

In [8]:
torch.cuda.empty_cache()

In [31]:
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
USE_GPU = True
TE_DTYPE = torch.float32
TARGET_DIM = 1
NUM_EPOCHS = 100

model = Net(TARGET_DIM)
image_data = WavImagesLoader('data/dataset.csv')
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loader_train = DataLoader(image_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=12)
image_trainer(loader_train, model, optimizer, exp_name='test', save_every=1, print_every=200, epochs=NUM_EPOCHS,
      use_gpu=USE_GPU, dtype=TE_DTYPE)

Epoch 0


RuntimeError: CUDA out of memory. Tried to allocate 1.21 GiB (GPU 1; 10.76 GiB total capacity; 5.62 GiB already allocated; 823.38 MiB free; 9.58 MiB cached)

In [9]:
!pwd

/projects/audio-pipelines/data


In [ ]:
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
USE_GPU = True
TE_DTYPE = torch.float32
TARGET_DIM = 1
NUM_EPOCHS = 100

model = Net(TARGET_DIM)
model.load_state_dict(torch.load('models/image_cnn_100e_2h.pt'))
model.train()
image_data = WavImagesLoader('data/dataset.csv')
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loader_train = DataLoader(image_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=12)
image_trainer(loader_train, model, optimizer, exp_name='image_cnn_100e', save_every=10, print_every=200, epochs=NUM_EPOCHS,
      use_gpu=USE_GPU, dtype=TE_DTYPE)

Epoch 0
Iteration 0, loss = 0.2754; epoch 0
Iteration 200, loss = 0.2162; epoch 0
Iteration 400, loss = 0.2357; epoch 0
Epoch 1
Iteration 0, loss = 0.1789; epoch 1
Iteration 200, loss = 0.2216; epoch 1
Iteration 400, loss = 0.2336; epoch 1
Epoch 2
Iteration 0, loss = 0.2543; epoch 2
Iteration 200, loss = 0.2484; epoch 2
Iteration 400, loss = 0.3190; epoch 2
Epoch 3
Iteration 0, loss = 0.2069; epoch 3
Iteration 200, loss = 0.2297; epoch 3
Iteration 400, loss = 0.2148; epoch 3
Epoch 4
Iteration 0, loss = 0.1457; epoch 4
Iteration 200, loss = 0.1440; epoch 4
Iteration 400, loss = 0.2731; epoch 4
Epoch 5
Iteration 0, loss = 0.2032; epoch 5
Iteration 200, loss = 0.2866; epoch 5
Iteration 400, loss = 0.2399; epoch 5
Epoch 6
Iteration 0, loss = 0.1370; epoch 6
Iteration 200, loss = 0.2796; epoch 6
Iteration 400, loss = 0.2460; epoch 6
Epoch 7
Iteration 0, loss = 0.2543; epoch 7
Iteration 200, loss = 0.1936; epoch 7
Iteration 400, loss = 0.1868; epoch 7
Epoch 8
Iteration 0, loss = 0.1928; epoc